In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import torch
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup

from tqdm import tqdm_notebook 

In [3]:
with open('mypersonality_final.csv', 'r', encoding='utf-8', errors='ignore') as file:
    data = pd.read_csv(file)

In [4]:
data.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


- Here we can see that if person have score greater than 3 then it is yes for that column else no.
- Also for same person we have multiple entries 
- so we have to take the mean of that partcular column and if the mean is greater than 3 then it is yes else no.

Here we saw that if person have ['sEXT', 'sNEU', 'sAGR', 'sCON', 'sOPN'] greater than 3 then it is yes for that column else no in ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
So we keep only ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN'] columns as they show if that personality is yes or no by that user.

In [8]:
data_trim = data[['STATUS', 'cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']]
data_trim.head()

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
0,likes the sound of thunder.,n,y,n,n,y
1,is so sleepy it's not even funny that's she ca...,n,y,n,n,y
2,is sore and wants the knot of muscles at the b...,n,y,n,n,y
3,likes how the day sounds in this new song.,n,y,n,n,y
4,is home. <3,n,y,n,n,y


In [9]:
# changing n to 0 
replacement_map = {'n': 0, 'y': 1}
columns_to_replace = ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
data_trim[columns_to_replace] = data_trim[columns_to_replace].replace(replacement_map)
data_trim.to_csv('data_trim.csv', index=False) # saving the table
data_trim.head()


/tmp/ipykernel_30704/344187970.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_trim[columns_to_replace] = data_trim[columns_to_replace].replace(replacement_map)


,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
0,likes the sound of thunder.,0,1,0,0,1
1,is so sleepy it's not even funny that's she ca...,0,1,0,0,1
2,is sore and wants the knot of muscles at the b...,0,1,0,0,1
3,likes how the day sounds in this new song.,0,1,0,0,1
4,is home. <3,0,1,0,0,1


Here we have 5 personallity 'Openness': 'cOPN', 'Conscientiousness': 'cCON', 'Extraversion': 'cEXT','Agreeableness': 'cAGR','Neuroticism': 'cNEU'
- Here we can see person can have multiple personality. So each one of them can be considered as a different output.
- So we divide the data into 5 parts and train 5 different models for each personality.

- We will do tfidf vectorization on the posts and then train the model using this vector as imput.

In [10]:
X1 = data_trim[['STATUS', 'cEXT']]
X1 

,STATUS,cEXT
0,likes the sound of thunder.,0
1,is so sleepy it's not even funny that's she ca...,0
2,is sore and wants the knot of muscles at the b...,0
3,likes how the day sounds in this new song.,0
4,is home. <3,0
...,...,...
9912,little things give you away.,0
9913,is wishing it was Saturday.,1
9914,is studying hard for the G.R.E.,1
9915,snipers get more head,0


In [13]:
def train_ml(dataset, column_name):
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

    X = tfidf_vectorizer.fit_transform(dataset['STATUS'])
    y = dataset[column_name]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    model = RandomForestClassifier(n_estimators=50, random_state=42) 
    model.fit(X_train, y_train)
    # Model Evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [18]:
x1_accuracy = train_ml(X1,'cEXT' )
print('Accuracy wrt personality Extraversion colum name cEXT',x1_accuracy)

X2 = data_trim[['STATUS', 'cNEU']]
x2_accuracy = train_ml(X2,'cNEU' )
print('Accuracy wrt personality Neuroticism colum name cNEU',x2_accuracy)

X3 = data_trim[['STATUS', 'cAGR']]
x3_accuracy = train_ml(X3,'cAGR' )
print('Accuracy wrt personality Agreeableness colum name cAGR',x3_accuracy)

X4 = data_trim[['STATUS', 'cEXT']]
x4_accuracy = train_ml(X4,'cEXT' )
print('Accuracy wrt personality Conscientiousness colum name cCON',x4_accuracy)

X5 = data_trim[['STATUS', 'cOPN']]
x5_accuracy = train_ml(X5,'cOPN' )
print('Accuracy wrt personality Openness colum name cOPN',x5_accuracy)


Accuracy wrt personality Extraversion colum name cEXT 0.5559475806451613
Accuracy wrt personality Neuroticism colum name cNEU 0.6199596774193549
Accuracy wrt personality Agreeableness colum name cAGR 0.5418346774193549
Accuracy wrt personality Conscientiousness colum name cCON 0.5559475806451613
Accuracy wrt personality Openness colum name cOPN 0.735383064516129


Above accuries using random forest is better than atleast selecting at random ie 50% accuracy.

# Lets try to improve the accuracy

In [19]:
def train_bert(dataset, epochs, column_name):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(dataset['STATUS'], dataset[column_name], test_size=0.2, random_state=42)

    # Load the pre-trained BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 for binary classification

    # Tokenize the text data and prepare input tensors for X_train
    max_length = 128

    input_ids_train = []
    attention_masks_train = []

    for text in X_train:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids_train.append(encoded_dict['input_ids'])
        attention_masks_train.append(encoded_dict['attention_mask'])

    input_ids_train = torch.cat(input_ids_train, dim=0)
    attention_masks_train = torch.cat(attention_masks_train, dim=0)

    # Create DataLoader for batch processing
    batch_size = 32
    train_dataset = TensorDataset(input_ids_train, attention_masks_train, torch.tensor(y_train.tolist()))
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Define optimizer and learning rate scheduler
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader))

    # Fine-tune the BERT model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    epochs = epochs  # You can adjust this based on your dataset and resources

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_dataloader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }

            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        average_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss:.4f}")
    # Model Evaluation
    model.eval()
    y_pred = []
    for text in X_test:
        inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_labels = torch.argmax(probabilities, dim=1)
            y_pred.extend(predicted_labels.cpu().numpy())

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


In [23]:
x1_accuracy_bert = train_bert(X1, 4, column_name = 'cEXT')
print('Accuracy wrt personality Extraversion colum name cEXT',x1_accuracy_bert)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

Epoch 1/4, Average Loss: 0.6772
Epoch 2/4, Average Loss: 0.6543
Epoch 3/4, Average Loss: 0.6537
Epoch 4/4, Average Loss: 0.6549
Accuracy wrt personality Extraversion colum name cEXT 0.5866935483870968


In [24]:
X2 = data_trim[['STATUS', 'cNEU']]
x2_accuracy_bert = train_bert(X2,epochs = 4, column_name = 'cNEU' )
print('Accuracy wrt personality Neuroticism colum name cNEU',x2_accuracy_bert) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

Epoch 1/4, Average Loss: 0.6609
Epoch 2/4, Average Loss: 0.6310
Epoch 3/4, Average Loss: 0.6326
Epoch 4/4, Average Loss: 0.6334
Accuracy wrt personality Neuroticism colum name cNEU 0.6320564516129032


In [25]:
X3 = data_trim[['STATUS', 'cAGR']]
x3_accuracy_bert = train_bert(X3,4, 'cAGR' )
print('Accuracy wrt personality Agreeableness colum name cAGR',x3_accuracy_bert)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

Epoch 1/4, Average Loss: 0.6947
Epoch 2/4, Average Loss: 0.6894
Epoch 3/4, Average Loss: 0.6854
Epoch 4/4, Average Loss: 0.6880
Accuracy wrt personality Agreeableness colum name cAGR 0.5650201612903226


In [26]:
X4 = data_trim[['STATUS', 'cEXT']]
x4_accuracy_bert = train_bert(X4,4,'cEXT' )
print('Accuracy wrt personality Conscientiousness colum name cCON',x4_accuracy_bert)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

Epoch 1/4, Average Loss: 0.6758
Epoch 2/4, Average Loss: 0.6590
Epoch 3/4, Average Loss: 0.6588
Epoch 4/4, Average Loss: 0.6581
Accuracy wrt personality Conscientiousness colum name cCON 0.5816532258064516


In [27]:
X5 = data_trim[['STATUS', 'cOPN']]
x5_accuracy_bert = train_bert(X5,4, 'cOPN' )
print('Accuracy wrt personality Openness colum name cOPN',x5_accuracy_bert)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

Epoch 1/4, Average Loss: 0.5623
Epoch 2/4, Average Loss: 0.5284
Epoch 3/4, Average Loss: 0.5291
Epoch 4/4, Average Loss: 0.5306
Accuracy wrt personality Openness colum name cOPN 0.7605846774193549


We can see using only 4 epochs we can achive better results than random forest. By training with more epochs we can attain better accuracy